In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, random_split
import numpy as np
from scipy import linalg
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns
import os
from tqdm import tqdm

# 设置随机种子以确保可复现性
torch.manual_seed(42)
np.random.seed(42)

In [2]:
# 1. 数据加载与预处理
def load_data():
    """加载并预处理MNIST数据集"""
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))  # MNIST数据集的均值和标准差
    ])
    
    # 加载训练集和测试集
    train_data = datasets.MNIST('data', train=True, download=True, transform=transform)
    test_data = datasets.MNIST('data', train=False, transform=transform)
    
    # 划分训练集和验证集
    train_size = int(0.8 * len(train_data))
    val_size = len(train_data) - train_size
    train_set, val_set = random_split(train_data, [train_size, val_size])
    
    return train_set, val_set, test_data

In [3]:
# # 2. 定义CNN模型
# class CNN(nn.Module):
#     """MNIST分类的卷积神经网络"""
#     def __init__(self):
#         super(CNN, self).__init__()
#         self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
#         self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
#         self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
#         self.fc1 = nn.Linear(64 * 7 * 7, 128)
#         self.fc2 = nn.Linear(128, 10) #对应MNIST的10个类别：0-9
#         self.dropout = nn.Dropout(0.25)
        
#     def forward(self, x):
#         x = self.pool(F.relu(self.conv1(x)))
#         x = self.pool(F.relu(self.conv2(x)))
#         x = x.view(-1, 64 * 7 * 7)  # 展平
#         x = self.dropout(F.relu(self.fc1(x)))
#         x = self.fc2(x)
#         return x

In [4]:
# 2. 定义4层卷积的CNN模型
class CNN(nn.Module):
    """具有4层卷积的MNIST分类网络"""
    def __init__(self, input_size, num_classes):
        super(CNN, self).__init__()
        # 第一组卷积+池化
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(32)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout1 = nn.Dropout(0.1)
        
        # 第二组卷积+池化
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(64)
        self.conv4 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.bn4 = nn.BatchNorm2d(128)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout2 = nn.Dropout(0.2)
        
        # 第三组卷积+池化
        self.conv5 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)
        self.bn5 = nn.BatchNorm2d(256)
        self.conv6 = nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1)
        self.bn6 = nn.BatchNorm2d(512)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2, padding=1)
        self.dropout3 = nn.Dropout(0.2)
        
        
        # 动态计算全连接层输入维度
        dummy_input = torch.rand(1, *input_size)  # 创建虚拟输入
        conv_output_size = self._get_conv_output_size(dummy_input)
        
        # 定义全连接部分
        self.fc1 = nn.Linear(conv_output_size, 1024)
        # self.fc1 = nn.Linear(512 * 4 * 4, 1024)
        self.dropout4 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(1024, num_classes)
    
    
    def _get_conv_output_size(self, x):
        """通过前向传播计算卷积部分的输出尺寸"""
        # 第一组卷积层
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.pool1(x)
        x = self.dropout1(x)
        
        # 第二组卷积层
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.relu(self.bn4(self.conv4(x)))
        x = self.pool2(x)
        x = self.dropout2(x)
        
        # 第三组卷积层
        x = F.relu(self.bn5(self.conv5(x)))
        x = F.relu(self.bn6(self.conv6(x)))
        x = self.pool3(x)
        x = self.dropout3(x)
        
        return x.view(1, -1).size(1)  # 返回展平后的维度
        
    
    def forward(self, x):
        # 第一组卷积层
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.pool1(x)
        x = self.dropout1(x)
        
        # 第二组卷积层
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.relu(self.bn4(self.conv4(x)))
        x = self.pool2(x)
        x = self.dropout2(x)
        
        # 第三组卷积层
        x = F.relu(self.bn5(self.conv5(x)))
        x = F.relu(self.bn6(self.conv6(x)))
        x = self.pool3(x)
        x = self.dropout3(x)
        
        # 展平并通过全连接层
        # x = x.view(-1, 1024 * 4 * 4)
        x = x.view(x.size(0), -1)  # 自动计算维度
        x = F.relu(self.fc1(x))
        x = self.dropout4(x)
        x = self.fc2(x)
        return x

In [5]:
# 3. 训练模型
def train(model, train_loader, val_loader, epochs, lr):
    """训练CNN模型"""
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    scheduler = StepLR(optimizer, step_size=10, gamma=0.5)
    
    train_losses = []
    val_losses = []
    val_accuracies = []
    best_val_loss = float('inf')
    best_model = None
    
    for epoch in range(epochs):
        # 训练阶段
        model.train()
        train_loss = 0
        for data, target in tqdm(train_loader, desc=f'Epoch {epoch+1}/{epochs} [Train]'):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        
        train_loss /= len(train_loader)
        train_losses.append(train_loss)
        
        # 验证阶段
        model.eval()
        val_loss = 0
        correct = 0
        total = 0
        with torch.no_grad():
            for data, target in tqdm(val_loader, desc=f'Epoch {epoch+1}/{epochs} [Val]'):
                data, target = data.to(device), target.to(device)
                output = model(data)
                loss = criterion(output, target)
                val_loss += loss.item()
                
                _, predicted = output.max(1)
                total += target.size(0)
                correct += predicted.eq(target).sum().item()
        
        val_loss /= len(val_loader)
        val_accuracy = 100. * correct / total
        val_losses.append(val_loss)
        val_accuracies.append(val_accuracy)
        
        print(f'Epoch {epoch+1}/{epochs}, '
              f'Train Loss: {train_loss:.4f}, '
              f'Val Loss: {val_loss:.4f}, '
              f'Val Acc: {val_accuracy:.2f}%')
        
        # 保存最佳模型
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model = model.state_dict().copy()
            print(f'Best model saved at epoch {epoch+1} with val loss: {val_loss:.4f}')
        
        scheduler.step()
    
    # 加载最佳模型
    model.load_state_dict(best_model)
    
    return model, train_losses, val_losses, val_accuracies

In [6]:
# 4. 评估模型
def evaluate(model, test_loader):
    """评估模型在测试集上的性能"""
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    model.eval()
    
    test_loss = 0
    correct = 0
    total = 0
    all_targets = []
    all_predictions = []
    
    criterion = nn.CrossEntropyLoss()
    
    with torch.no_grad():
        for data, target in tqdm(test_loader, desc='Evaluating'):
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = criterion(output, target)
            test_loss += loss.item()
            
            _, predicted = output.max(1)
            total += target.size(0)
            correct += predicted.eq(target).sum().item()
            
            all_targets.extend(target.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())
    
    test_loss /= len(test_loader)
    test_accuracy = 100. * correct / total
    
    print(f'Test Loss: {test_loss:.4f}, Test Acc: {test_accuracy:.2f}%')
    
    # 计算混淆矩阵
    cm = confusion_matrix(all_targets, all_predictions)
    
    return test_accuracy, cm

In [7]:
# 5. 生成样本函数（使用简单的噪声扰动）
def generate_samples(model, num_samples=5000, device='cpu'):
    """使用训练好的模型生成样本"""
    model = model.to(device)
    model.eval()
    
    # 从测试集随机选择样本并添加噪声
    _, _, test_data = load_data()
    test_loader = DataLoader(test_data, batch_size=num_samples, shuffle=True)
    
    # 获取一批测试数据
    data, _ = next(iter(test_loader))
    data = data[:num_samples].to(device)
    
    # 添加随机噪声生成新样本
    generated_samples = data + torch.randn_like(data) * 0.1
    generated_samples = torch.clamp(generated_samples, -1, 1)
    
    return generated_samples

In [8]:
# 6. FID计算函数
def calculate_frechet_distance(mu1, sigma1, mu2, sigma2, eps=1e-6):
    """计算两个分布之间的Frechet距离"""
    # 转换为numpy数组
    mu1 = np.atleast_1d(mu1)
    mu2 = np.atleast_1d(mu2)
    
    sigma1 = np.atleast_2d(sigma1)
    sigma2 = np.atleast_2d(sigma2)
    
    assert mu1.shape == mu2.shape, f"Mean vectors have different lengths: {mu1.shape}, {mu2.shape}"
    assert sigma1.shape == sigma2.shape, f"Covariances have different dimensions: {sigma1.shape}, {sigma2.shape}"
    
    # 计算Frechet距离
    diff = mu1 - mu2
    
    # 计算矩阵平方根
    covmean, _ = linalg.sqrtm(sigma1.dot(sigma2), disp=False)
    
    # 处理非正定矩阵的情况
    if not np.isfinite(covmean).all():
        msg = ("fid calculation produces singular product; "
               "adding %s to diagonal of cov estimates") % eps
        print(msg)
        offset = np.eye(sigma1.shape[0]) * eps
        covmean = linalg.sqrtm((sigma1 + offset).dot(sigma2 + offset))
    
    # 检查是否有虚数部分
    if np.iscomplexobj(covmean):
        if not np.allclose(np.diagonal(covmean).imag, 0, atol=1e-3):
            m = np.max(np.abs(covmean.imag))
            raise ValueError(f"Imaginary component {m}")
        covmean = covmean.real
    
    tr_covmean = np.trace(covmean)
    
    return (diff.dot(diff) + np.trace(sigma1) + np.trace(sigma2) - 2 * tr_covmean)

def calculate_activation_statistics(images, model, batch_size=128, device='cpu'):
    """计算激活统计量（均值和协方差矩阵）"""
    model.eval()
    
    if batch_size > len(images):
        print(f"Warning: Batch size is bigger than the data size. Setting batch size to {len(images)}")
        batch_size = len(images)
    
    dl = DataLoader(images, batch_size=batch_size)
    
    
    #先通过一个样本确定特征维度（方法1）
    with torch.no_grad():
        sample = images[0:1].to(device)
        pred = model(sample)
        if len(pred.shape) > 2:
            pred = pred.reshape(pred.size(0), -1)
        dims = pred.size(1)
        
    # dims = 128 * 7 * 7 #参考CNN模型结构定义部分（方法2）
    pred_arr = np.empty((len(images), dims))
    
    start_idx = 0
    
    for batch in tqdm(dl, desc='Calculating activation statistics'):
        batch = batch.to(device)
        
        with torch.no_grad():
            pred = model(batch)
        
        # 如果输出不是2D，则展平
        if len(pred.shape) > 2:
            pred = pred.reshape(pred.size(0), -1)
        
        pred_arr[start_idx:start_idx + pred.size(0)] = pred.cpu().numpy()
        
        start_idx += pred.size(0)
    
    mu = np.mean(pred_arr, axis=0)
    sigma = np.cov(pred_arr, rowvar=False)
    
    return mu, sigma

In [9]:
# 7. 特征提取模型（简化版）
class FeatureExtractor(nn.Module):
    """用于FID计算的特征提取器"""
    def __init__(self, model):
        super(FeatureExtractor, self).__init__()
        # 使用训练好的CNN的特征提取部分
        self.features = nn.Sequential(
            model.conv1,
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            model.conv2,
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)  # 展平为特征向量
        return x

In [10]:
# 8. 可视化函数
def plot_training_history(train_losses, val_losses, val_accuracies):
    """绘制训练历史"""
    plt.figure(figsize=(12, 4))
    
    plt.subplot(1, 2, 1)
    plt.plot(train_losses, label='Train Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss')
    plt.legend()
    plt.grid(True)
    
    plt.subplot(1, 2, 2)
    plt.plot(val_accuracies, label='Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy (%)')
    plt.title('Validation Accuracy')
    plt.legend()
    plt.grid(True)
    
    plt.tight_layout()
    plt.savefig('training_history.png')
    plt.close()

def plot_confusion_matrix(cm):
    """绘制混淆矩阵"""
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=list(range(10)), yticklabels=list(range(10)))
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.title('Confusion Matrix')
    plt.savefig('confusion_matrix.png')
    plt.close()

def save_samples(samples, num_samples=25, filename='generated_samples.png'):
    """保存生成的样本图像"""
    plt.figure(figsize=(10, 10))
    
    for i in range(num_samples):
        plt.subplot(5, 5, i+1)
        plt.imshow(samples[i][0].cpu().numpy(), cmap='gray')
        plt.axis('off')
    
    plt.tight_layout()
    plt.savefig(filename)
    plt.close()

In [11]:
# 9. 主函数
def main(input_size=(1, 28, 28), num_classes=10, epochs=50, lr=0.0001):
    # 加载数据
    train_set, val_set, test_data = load_data()
    
    # 创建数据加载器
    train_loader = DataLoader(train_set, batch_size=128, shuffle=True)
    val_loader = DataLoader(val_set, batch_size=128)
    test_loader = DataLoader(test_data, batch_size=128)
    
    # 创建模型
    model = CNN(input_size, num_classes)
    
    # 训练模型
    model, train_losses, val_losses, val_accuracies = train(model, train_loader, val_loader, epochs, lr)
    
    # 评估模型
    test_accuracy, cm = evaluate(model, test_loader)
    
    # 生成样本
    generated_samples = generate_samples(model, num_samples=5000)
    
    # 保存部分生成的样本
    save_samples(generated_samples, num_samples=25, filename='generated_samples.png')
    
    # 计算FID
    feature_extractor = FeatureExtractor(model)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    feature_extractor = feature_extractor.to(device)
    
    # 计算真实样本的统计量
    real_images = torch.stack([test_data[i][0] for i in range(5000)]).to(device)
    mu_real, sigma_real = calculate_activation_statistics(real_images, feature_extractor, device=device)
    
    # 计算生成样本的统计量
    mu_gen, sigma_gen = calculate_activation_statistics(generated_samples, feature_extractor, device=device)
    
    # 计算FID
    fid = calculate_frechet_distance(mu_real, sigma_real, mu_gen, sigma_gen)
    print(f"FID Score: {fid:.4f}")
    
    # 可视化训练历史和混淆矩阵
    plot_training_history(train_losses, val_losses, val_accuracies)
    plot_confusion_matrix(cm)
    
    # 保存模型
    torch.save(model.state_dict(), 'mnist_cnn_model.pth')
    print("Model saved as 'mnist_cnn_model.pth'")

In [12]:
if __name__ == "__main__":
    main(input_size=(1, 28, 28), num_classes=10, epochs=50, lr=0.0001)

Epoch 1/50 [Val]: 100%|██████████| 94/94 [00:01<00:00, 91.49it/s]


Epoch 1/50, Train Loss: 0.2497, Val Loss: 0.0538, Val Acc: 98.27%
Best model saved at epoch 1 with val loss: 0.0538


Epoch 2/50 [Val]: 100%|██████████| 94/94 [00:01<00:00, 93.90it/s]


Epoch 2/50, Train Loss: 0.0607, Val Loss: 0.0413, Val Acc: 98.72%
Best model saved at epoch 2 with val loss: 0.0413


Epoch 3/50 [Val]: 100%|██████████| 94/94 [00:01<00:00, 93.81it/s]


...

Epoch 50/50, Train Loss: 0.0007, Val Loss: 0.0239, Val Acc: 99.51%


Evaluating: 100%|██████████| 79/79 [00:00<00:00, 93.12it/s]


Test Loss: 0.0179, Test Acc: 99.60%


Calculating activation statistics: 100%|██████████| 40/40 [00:00<00:00, 1405.75it/s]


FID Score: 29.4342
Model saved as 'mnist_cnn_model.pth'
